# Surf's Up 
## Unit 10: Advanced SQL and APIs

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# pd.options.display.max_rows = 3000

# create engine for connections/sessions
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
# build inspector (used throughout)
inspector = inspect(engine)

In [ ]:
# gather keys for data
# Base.classes.keys()

## this is functionally the same as
# inspector = inspect(engine)
# inspector.get_table_names()

# RESULTS: measurement, station

In [ ]:
# inspect station data (specifically looking for column names...)

# Station_info = inspector.get_columns('station')
# for header in Station_info:
#     print(header['name'])

## RESULTS: id, station, name, latitude, longitude, elevation

In [ ]:
# inspect measurement data (specifically looking for column names...)

# Measurement_info = inspector.get_columns('measurement')
# for header in Measurement_info:
#     print(header['name'])

## RESULTS: id, station, date, prcp, tobs

In [ ]:
# reflect tables into classes
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
session = Session(engine)

In [ ]:
conn = engine.connect()

## Precipitation Analysis

In [ ]:
# read data into df, convert date from string to usable datetime
precipitation_df = pd.read_sql_query('SELECT "date", "prcp" FROM measurement', conn)

In [ ]:
precipitation_df['date'] = pd.to_datetime(precipitation_df['date'])
end = precipitation_df['date'].max()
start = end - dt.timedelta(days=364)

In [ ]:
mask = (precipitation_df['date'] >= start) & (precipitation_df['date'] <= end)
twelve_mo_precip_df = precipitation_df.loc[mask]
# twelve_mo_df

In [ ]:
precip_df = twelve_mo_precip_df.groupby('date')
plotting_precip_df = precip_df.sum()
# plotting_precip_df

In [ ]:
# plot daily precipitation totals from the dataframe

plotting_precip_df.plot(kind='bar', figsize=(20, 15))

plt.title('Daily Precipitation Totals', fontsize = 45)
plt.xlabel('Dates', fontsize = 30)
plt.ylabel('Precipitation (inches)', fontsize = 30)
plt.yticks(fontsize = 20)

plt.tick_params(
    axis='x', 
    which='both', 
    bottom=False, 
    top=False, 
    labelbottom=False) 

legend = plt.legend()
legend.remove()

plt.tight_layout()
plt.show()

# REVISIT: figure out how to put in periodic x-ticks with corresponding labels
# 365 bars is too many for *good* visualization if you are trying to *use* it to make a decision..
# If just for seeing the 'shape' it's fine...

In [ ]:
# print summary stats for daily precipitation
plotting_precip_df.describe()

## Station Analysis

### Total Number of Stations

In [ ]:
# query for total number of stations
station_count = session.query(Station.name).count()
print(f'There are {station_count} participating stations.')

### Stations by Observation Count

In [ ]:
station_activity = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).\
    desc())

for station in station_activity:
    print(station)

### Station with Most Observations

In [ ]:
station_activity_max = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).desc()).\
    first()

print(f'Station: {station_activity_max[0]}')
print(f'Observation Count: {station_activity_max[1]}')

In [ ]:
temperature_df = pd.read_sql_query('SELECT "station", "date", "tobs" FROM measurement', conn)

In [ ]:
temperature_df['date'] = pd.to_datetime(temperature_df['date'])
end = temperature_df['date'].max()
start = end - dt.timedelta(days=364)

In [ ]:
mask = (temperature_df['date'] >= start) & (temperature_df['date'] <= end)
twelve_mo_temp_df = temperature_df.loc[mask]
# twelve_mo_temp_df

In [ ]:
temp_df = twelve_mo_temp_df.groupby('station')
plotting_temp_df = temp_df.count()
plotting_temp_df

In [ ]:
# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
# save and display image
plt.savefig('Images/ <NAME HERE> .png')
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
# save and display image
plt.savefig('Images/ <NAME HERE> .png')
plt.show()